In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchvision.transforms as transforms

# Add the parent directory to the Python path - bad practice, but it's just for the example
import sys
import sys
project_root = "/home/heydari/FHHI-XAI/"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
import random
import copy
import logging
import re
import pandas as pd
from LCRP.utils.crp_configs import ATTRIBUTORS, CANONIZERS, COMPOSITES
from src.glocal_analysis import run_analysis
from src.plot_crp_explanations import plot_explanations
from src.datasets.person_car_dataset import PersonCarDataset
from LCRP.models import get_model
import LCRP.models.yolov6 as yolov6


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [5]:
device

'cuda:0'

In [6]:

dtype = torch.float32
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Resize((1280, 1280)),
    transforms.Lambda(lambda x: x.to(dtype)), 
])


# Load dataset
root_dir = "../src/datasets/data/person_car_detection_data/Arthal/"
dataset = PersonCarDataset(root_dir=root_dir, split="train", transform=transform)

model_name = "yolov6s6"
ckpt_path = "../models/best_v6s6_ckpt.pt"

# Loading unet with path to checkpoint
model = get_model(model_name=model_name, classes=2, ckpt_path=ckpt_path, device=device, dtype=dtype)

In [15]:
model.to(device)


YoloV6S6_Model(
  (module): Model(
    (backbone): EfficientRep6(
      (stem): RepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (rbr_dense): ConvModule(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        )
        (rbr_1x1): ConvModule(
          (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        )
      )
      (ERBlock_2): Sequential(
        (0): RepVGGBlock(
          (nonlinearity): ReLU(inplace=True)
          (se): Identity()
          (rbr_dense): ConvModule(
            (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          )
      

In [7]:

# === Setup ===
n_samples = 100
dataset_size = len(dataset)  # Your dataset must be loaded here
n_concepts = 3
class_id = 1
model_name = "yolov6s6"


yolo_layers = [
    "module.backbone.stem.rbr_dense.conv",
    "module.backbone.ERBlock_2.0.rbr_dense.conv",
    "module.backbone.ERBlock_3.1.block.0.rbr_dense.conv",
    "module.backbone.ERBlock_4.1.block.1.rbr_1x1.conv",
    "module.neck.Rep_p3.conv1.rbr_1x1.conv",
    "module.neck.Rep_p5.block.2.rbr_dense.conv"


]

log_file_path = "../examples/log_yolo_test.txt"

# === Configure Logging ===
logging.basicConfig(filename=log_file_path, level=logging.ERROR, force=True)

# === Attribution Loop ===
sample_indices = random.sample(range(dataset_size), n_samples)

for i, sample_idx in enumerate(sample_indices):
    layer_idx = i % len(yolo_layers)
    layer = yolo_layers[layer_idx]
    
    image_tensor, _ = dataset[sample_idx]
    
    # Log sample & layer before attribution
    logging.error(f"SAMPLE {sample_idx}, LAYER {layer}")
    
    attribution = ATTRIBUTORS[model_name](model)
    composite = COMPOSITES[model_name](canonizers=[CANONIZERS[model_name]()])
    condition = [{"y": class_id}]
    
    attr = attribution(
        copy.deepcopy(image_tensor).unsqueeze(0).requires_grad_(),
        condition,
        composite,
        record_layer=[layer],
        init_rel=1
    )

print("Attribution complete, logs saved.")

# === Parsing Logs & Building Tables ===
with open(log_file_path, 'r') as f:
    logs = f.readlines()

results = []
sample_id = None
layer_name = None

for line in logs:
    sample_match = re.match(r'.*SAMPLE (\d+), LAYER (.+)', line)
    if sample_match:
        sample_id = int(sample_match.group(1))
        layer_name = sample_match.group(2)
    time_match = re.search(r'Prediction time: ([\d\.]+), Backward time: ([\d\.]+), Full attribution time: ([\d\.]+)', line)
    if time_match and sample_id is not None:
        results.append({
            "Sample ID": sample_id,
            "Layer": layer_name,
            "Prediction Time (s)": float(time_match.group(1)),
            "Backward Time (s)": float(time_match.group(2)),
            "Full Attribution Time (s)": float(time_match.group(3))
        })
        sample_id, layer_name = None, None  # Reset after match

# === Save Detailed Sample Results ===
df = pd.DataFrame(results)
df = df.sort_values(by="Sample ID")  # Sort for readability
df.to_csv("detailed_sample_times.csv", index=False)

# === Compute & Save Global Summary Table ===
avg_pred_time = df["Prediction Time (s)"].mean()
global_lcrp_time = df["Full Attribution Time (s)"].sum()
attr_single = df["Backward Time (s)"].mean()
allowed_time = attr_single * dataset_size

summary_table = pd.DataFrame({
    "Metric": [
        "Model Prediction (average per sample)",
        "Global (LCRP) Total Time",
        "Attribution Time per Sample",
        "Allowed Time for Global Explanation",
        "Number of Data Points"
    ],
    "Value": [
        f"{avg_pred_time:.4f} s",
        f"{global_lcrp_time:.2f} s",
        f"{attr_single:.4f} s",
        f"{allowed_time:.2f} s",
        f"{dataset_size}"
    ]
})

summary_table.to_csv("KPI_explanation_summary.csv", index=False)

# === Display Output ===
print("\n==== Global Explanation Summary ====")
print(summary_table.to_string(index=False))
print("Results saved to 'detailed_sample_times.csv' and 'KPI_explanation_summary.csv'")


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [9]:
import pandas as pd

# Load the detailed sample times (produced by the main script)
df = pd.read_csv("detailed_sample_times.csv")

# Calculate the Backward / Prediction Time Ratio per sample
df["Backward/Prediction Ratio"] = df["Backward Time (s)"] / df["Prediction Time (s)"]
avg_ratio = df["Backward/Prediction Ratio"].mean()

print(f"Average Backward Time / Prediction Time Ratio: {avg_ratio:.2f}")

# Load summary table to append the new metric
summary_table = pd.read_csv("global_explanation_summary.csv")

# Append the ratio to the summary table
summary_table = pd.concat([
    summary_table,
    pd.DataFrame({
        "Metric": ["Average Backward/Prediction Time Ratio"],
        "Value": [f"{avg_ratio:.2f}"]
    })
], ignore_index=True)

# Save updated summary
summary_table.to_csv("global_explanation_summary.csv", index=False)

print("Summary table updated with ratio.")


Average Backward Time / Prediction Time Ratio: 9.04
Summary table updated with ratio.


In [ ]:
# Load detailed per-sample times
df = pd.read_csv("detailed_sample_times.csv")

# Calculate Backward / Prediction ratio per sample
df["Backward/Prediction Ratio"] = df["Backward Time (s)"] / df["Prediction Time (s)"]

# Calculate overall average ratio across all samples
avg_ratio = df["Backward/Prediction Ratio"].mean()

# Group by layer to compute average ratio per layer
layer_avg_ratios = df.groupby("Layer")["Backward/Prediction Ratio"].mean()

# Find the layer with the minimum average ratio
min_layer = layer_avg_ratios.idxmin()
min_ratio = layer_avg_ratios.min()

# === Final Output ===
print("Average Backward Time / Prediction Time Ratio (All Samples):")
print(f"{avg_ratio:.2f}")

print("Layer with Minimum Average Backward/Prediction Time Ratio:")
print(f"Layer: {min_layer}")
print(f"Average Ratio: {min_ratio:.2f}")

print("nAll Layers' Average Ratios (sorted):")
print(layer_avg_ratios.sort_values())


Average Backward Time / Prediction Time Ratio (All Samples):
9.04
Layer with Minimum Average Backward/Prediction Time Ratio:
Layer: module.backbone.ERBlock_2.0.rbr_dense.conv
Average Ratio: 8.98
nAll Layers' Average Ratios (sorted):
Layer
module.backbone.ERBlock_2.0.rbr_dense.conv    8.979345
module.neck.Rep_p4.conv1.rbr_dense.conv       9.060862
module.backbone.stem.rbr_dense.conv           9.067772
Name: Backward/Prediction Ratio, dtype: float64


In [1]:
!pkill -u heydari -f jupyter

In [2]:
!pkill -u said -f jupyter